In [16]:
#i "nuget: C:/Users/ncgui/Desktop/PointProcessDecoder/artifacts/package/release"

Restore sources C:/Users/ncgui/Desktop/PointProcessDecoder/artifacts/package/release

In [17]:
#r "nuget: TorchSharp"
#r "nuget: PointProcessDecoder.Core"
#r "nuget: PointProcessDecoder.Cpu"
#r "nuget: PointProcessDecoder.Plot"
#r "nuget: PointProcessDecoder.Simulation"
#r "nuget: OxyPlot.Core, 2.1.2"

Restore sources C:/Users/ncgui/Desktop/PointProcessDecoder/artifacts/package/release Installed Packages OxyPlot.Core, 2.1.2 PointProcessDecoder.Core, 1.0.0 PointProcessDecoder.Cpu, 1.0.0 PointProcessDecoder.Plot, 1.0.0 PointProcessDecoder.Simulation, 1.0.0 TorchSharp, 0.105.0

In [18]:
using System;
using System.IO;

using TorchSharp;
using static TorchSharp.torch;

using PointProcessDecoder.Core;
using PointProcessDecoder.Plot;
using PointProcessDecoder.Simulation;
using PointProcessDecoder.Core.Estimation;
using PointProcessDecoder.Core.Transitions;
using PointProcessDecoder.Core.Encoder;
using PointProcessDecoder.Core.Decoder;
using PointProcessDecoder.Core.StateSpace;
using PointProcessDecoder.Core.Likelihood;

In [19]:
string EncodeToHtml(byte[] data)
{
    return $"<img src=\"data:image/png;base64,{Convert.ToBase64String(data)}\">";
}

static Tensor ReadBinaryFile(
    string binary_file,
    Device device = null,
    ScalarType scalarType = ScalarType.Float32
)
{
    device ??= CPU;
    byte[] fileBytes = File.ReadAllBytes(binary_file);
    int elementCount = fileBytes.Length / sizeof(double);
    double[] doubleArray = new double[elementCount];
    Buffer.BlockCopy(fileBytes, 0, doubleArray, 0, fileBytes.Length);
    Tensor t = tensor(doubleArray, device: device, dtype: scalarType);
    return t;
}

public static (Tensor, Tensor) InitializeRealData(
    string positionFile,
    string marksFile,
    Device device = null,
    ScalarType scalarType = ScalarType.Float32
)
{
    var position = ReadBinaryFile(positionFile, device, scalarType);
    var marks = ReadBinaryFile(marksFile, device, scalarType);
    return (position, marks);
}

Heatmap PlotPrediction(
    Tensor prediction,
    Tensor position2D,
    string title,
    double xMin,
    double xMax,
    double yMin,
    double yMax,
    string outputDirectory
)
{
    Heatmap plotPrediction = new(
        xMin,
        xMax,
        yMin,
        yMax,
        title: title
    );

    plotPrediction.OutputDirectory = Path.Combine(plotPrediction.OutputDirectory, outputDirectory);
    plotPrediction.Show<float>(
        prediction,
        position2D
    );
    plotPrediction.Save(png: true);
    
    return plotPrediction;
}

In [20]:
string positionFile = "../data/positions_2D.bin";
string marksFile = "../data/marks.bin";

Device device = CPU;
ScalarType scalarType = ScalarType.Float32;
int markDimensions = 4;
int markChannels = 28;

var (position, marks) = InitializeRealData(
    positionFile: positionFile,
    marksFile: marksFile,
    device: device,
    scalarType: scalarType
);

position = position.reshape(-1, 2);
marks = marks.reshape(position.shape[0], markDimensions, markChannels);

In [21]:
position = position[TensorIndex.Slice(0, 100000)];
marks = marks[TensorIndex.Slice(0, 100000)];

In [22]:
var pointProcessModel = new PointProcessModel(
    estimationMethod: EstimationMethod.KernelCompression,
    transitionsType: TransitionsType.RandomWalk,
    encoderType: EncoderType.ClusterlessMarkEncoder,
    decoderType: DecoderType.StateSpaceDecoder,
    stateSpaceType: StateSpaceType.DiscreteUniformStateSpace,
    likelihoodType: LikelihoodType.Clusterless,
    minStateSpace: [0, 0],
    maxStateSpace: [120, 120],
    stepsStateSpace: [50, 50],
    observationBandwidth: [5, 5],
    stateSpaceDimensions: 2,
    markDimensions: markDimensions,
    markChannels: markChannels,
    markBandwidth: [1, 1, 1, 1],
    distanceThreshold: 1.5,
    sigmaRandomWalk: 1,
    device: device,
    scalarType: scalarType
);

# Encode

In [23]:
double fractionTraining = 0.8;
int trainingBatchSize = 1000;
int testingBatchSize = 60;

In [24]:
int nTraining = (int)(position.shape[0] * fractionTraining);

for (int i = 0; i < nTraining; i += trainingBatchSize)
{
    Console.WriteLine($"Training batch {i / trainingBatchSize + 1} of {nTraining / trainingBatchSize + 1}");
    var end = Math.Min(i + trainingBatchSize, nTraining);
    pointProcessModel.Encode(
        position[TensorIndex.Slice(i, end)],
        marks[TensorIndex.Slice(i, end)]
    );
}

Training batch 1 of 81
Training batch 2 of 81
Training batch 3 of 81
Training batch 4 of 81
Training batch 5 of 81
Training batch 6 of 81
Training batch 7 of 81
Training batch 8 of 81
Training batch 9 of 81
Training batch 10 of 81
Training batch 11 of 81
Training batch 12 of 81
Training batch 13 of 81
Training batch 14 of 81
Training batch 15 of 81
Training batch 16 of 81
Training batch 17 of 81
Training batch 18 of 81
Training batch 19 of 81
Training batch 20 of 81
Training batch 21 of 81
Training batch 22 of 81
Training batch 23 of 81
Training batch 24 of 81
Training batch 25 of 81
Training batch 26 of 81
Training batch 27 of 81
Training batch 28 of 81
Training batch 29 of 81
Training batch 30 of 81
Training batch 31 of 81
Training batch 32 of 81
Training batch 33 of 81
Training batch 34 of 81
Training batch 35 of 81
Training batch 36 of 81
Training batch 37 of 81
Training batch 38 of 81
Training batch 39 of 81
Training batch 40 of 81
Training batch 41 of 81
Training batch 42 of 81
T

# Decode

In [25]:
int nTesting = (int)position.shape[0] - nTraining;
for (int i = nTraining; i < nTraining + nTesting; i += testingBatchSize)
{
    Console.WriteLine($"Testing batch {(i - nTraining) / testingBatchSize + 1} of {nTesting / testingBatchSize + 1}"); 
    var end = Math.Min(i + testingBatchSize, nTraining + nTesting);
    var prediction = pointProcessModel.Decode(marks[TensorIndex.Slice(i, end)]);
    prediction = (prediction.sum(dim: 0) / prediction.sum()).reshape([50, 50]);
    var title = $"Prediction2D_{i}-{end}";
    var positionSampled = position[TensorIndex.Slice(i, end)];
    var heatmap = PlotPrediction(
        prediction, 
        positionSampled, 
        title,
        0,
        120,
        0,
        120,
        "ClusterlessMark2D"
    );
}

Testing batch 1 of 334
Testing batch 2 of 334
Testing batch 3 of 334
Testing batch 4 of 334
Testing batch 5 of 334
Testing batch 6 of 334
Testing batch 7 of 334
Testing batch 8 of 334
Testing batch 9 of 334
Testing batch 10 of 334
Testing batch 11 of 334
Testing batch 12 of 334
Testing batch 13 of 334
Testing batch 14 of 334
Testing batch 15 of 334
Testing batch 16 of 334
Testing batch 17 of 334
Testing batch 18 of 334
Testing batch 19 of 334
Testing batch 20 of 334
Testing batch 21 of 334
Testing batch 22 of 334
Testing batch 23 of 334
Testing batch 24 of 334
Testing batch 25 of 334
Testing batch 26 of 334
Testing batch 27 of 334
Testing batch 28 of 334
Testing batch 29 of 334
Testing batch 30 of 334
Testing batch 31 of 334
Testing batch 32 of 334
Testing batch 33 of 334
Testing batch 34 of 334
Testing batch 35 of 334
Testing batch 36 of 334
Testing batch 37 of 334
Testing batch 38 of 334
Testing batch 39 of 334
Testing batch 40 of 334
Testing batch 41 of 334
Testing batch 42 of 334
T